In [1]:
import pandas as pd
import time

# Leitura dos Dados

In [2]:
# Dataset read
df = pd.read_csv('data/unpacked/ca-2019-2022.csv', delimiter=';')
# Types treatment
df['Data da Coleta'] =  pd.to_datetime(df['Data da Coleta'], format='%d/%m/%Y')
df['Valor de Venda'] = pd.to_numeric(df['Valor de Venda'].str.replace(',','.'))
df['Valor de Compra'] = pd.to_numeric(df['Valor de Compra'])#.str.replace(',','.'))
df.head(5)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,PE,IGARASSU,POSTO TREZE LTDA.,24.436.669/0001-99,RODOVIA BR 101 NORTE,250,KM 29,CENTRO,53610-296,DIESEL S10,2021-06-27,4.390,NaN,R$ / litro,BRANCA
1,NE,PE,IGARASSU,BARROS & ALMEIDA LTDA,24.140.204/0001-96,RODOVIA BR 101 NORTE,SN,KM 43498,CENTRO,53610-296,ETANOL,2021-06-27,5.199,NaN,R$ / litro,VIBRA ENERGIA
2,NE,PE,IGARASSU,BARROS & ALMEIDA LTDA,24.140.204/0001-96,RODOVIA BR 101 NORTE,SN,KM 43498,CENTRO,53610-296,GASOLINA,2021-06-27,5.799,NaN,R$ / litro,VIBRA ENERGIA
3,NE,PE,IGARASSU,BARROS & ALMEIDA LTDA,24.140.204/0001-96,RODOVIA BR 101 NORTE,SN,KM 43498,CENTRO,53610-296,GASOLINA ADITIVADA,2021-06-27,5.999,NaN,R$ / litro,VIBRA ENERGIA
4,NE,PE,IGARASSU,BARROS & ALMEIDA LTDA,24.140.204/0001-96,RODOVIA BR 101 NORTE,SN,KM 43498,CENTRO,53610-296,GNV,2021-06-27,3.599,NaN,R$ / m³,VIBRA ENERGIA


# Parâmetros do Experimento

In [3]:
# Number of iterations to calculate mean and std
n = 30

In [4]:
import numpy as np
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# Experimento Banco de Dados Relacional (SQLite)

In [5]:
from sqlalchemy import create_engine, Column, String, Integer, ForeignKey, DateTime, Float
from sqlalchemy.orm import Session

# Use the default method for abstracting classes to tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [6]:
class Coleta(Base):
    __tablename__ = 'coleta'
    id = Column(Integer, primary_key=True)
    regiao = Column(String(255))
    estado = Column(String(255))
    municipio = Column(String(255))
    revenda = Column(String(255))
    cnpj = Column(String(255))
    rua = Column(String(255))
    numero = Column(String(255))
    complemento = Column(String(255))
    bairro = Column(String(255))
    cep = Column(String(255))
    produto = Column(String(255))
    data = Column(DateTime())
    valor_venda = Column(Float(5))
    valor_compra = Column(Float(5))
    unidade_medida = Column(String(255))
    bandeira = Column(String(255))

In [7]:
#Insert and Delete Experiment
#SQLite

insertSQLite = []
deleteSQLite = []

for i in range(n):
    # Create DB connection
    engine = create_engine("sqlite:///precos.sqlite")
    conn = engine.connect()
    session = Session(bind=engine)

    # Create metadata layer that abstracts our SQL DB
    Base.metadata.create_all(engine)
    
    #Create objects
    tempo_ini = time.time()
    for index, row in df.iterrows():
        session.add(Coleta(
            id = index,
            regiao = row['Regiao - Sigla'],
            estado = row['Estado - Sigla'],
            municipio = row['Municipio'],
            revenda = row['Revenda'],
            cnpj = row['CNPJ da Revenda'],
            rua = row['Nome da Rua'],
            numero = row['Numero Rua'],
            complemento = row['Complemento'],
            bairro = row['Bairro'],
            cep = row['Cep'],
            produto = row['Produto'],
            data = row['Data da Coleta'],
            valor_venda = row['Valor de Venda'],
            valor_compra = row['Valor de Compra'],
            unidade_medida = row['Unidade de Medida'],
            bandeira = row['Bandeira']
        ))
    # Commit changes to database
    session.commit()
    tempo_fim = time.time()
    print(i)
    print('### SQL INSERT ### ', df.shape[0], 'registros.')
    print("Tempo de inserção: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    insertSQLite.append(tempo_fim - tempo_ini)

    # Delete database
    tempo_ini = time.time()
    Base.metadata.drop_all(engine)
    tempo_fim = time.time()
    print('### SQL DELETE ### ')
    print("Tempo de delete: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    deleteSQLite.append(tempo_fim - tempo_ini)




0
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 168.54610 segundos
### SQL DELETE ### 
Tempo de delete: 0.17375 segundos
1
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 167.37930 segundos
### SQL DELETE ### 
Tempo de delete: 0.17522 segundos
2
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 167.15228 segundos
### SQL DELETE ### 
Tempo de delete: 0.17400 segundos
3
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 166.85440 segundos
### SQL DELETE ### 
Tempo de delete: 0.17100 segundos
4
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 167.08009 segundos
### SQL DELETE ### 
Tempo de delete: 0.16900 segundos
5
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 167.60221 segundos
### SQL DELETE ### 
Tempo de delete: 0.17300 segundos
6
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 166.88989 segundos
### SQL DELETE ### 
Tempo de delete: 0.17100 segundos
7
### SQL INSERT ###  1011250 registros.
Tempo de inserção: 168.38513

In [8]:
# Create DB connection
engine = create_engine("sqlite:///precos.sqlite")
conn = engine.connect()
session = Session(bind=engine)

# Create metadata layer that abstracts our SQL DB
Base.metadata.create_all(engine)

#Create objects
tempo_ini = time.time()
for index, row in df.iterrows():
    session.add(Coleta(
        id = index,
        regiao = row['Regiao - Sigla'],
        estado = row['Estado - Sigla'],
        municipio = row['Municipio'],
        revenda = row['Revenda'],
        cnpj = row['CNPJ da Revenda'],
        rua = row['Nome da Rua'],
        numero = row['Numero Rua'],
        complemento = row['Complemento'],
        bairro = row['Bairro'],
        cep = row['Cep'],
        produto = row['Produto'],
        data = row['Data da Coleta'],
        valor_venda = row['Valor de Venda'],
        valor_compra = row['Valor de Compra'],
        unidade_medida = row['Unidade de Medida'],
        bandeira = row['Bandeira']
    ))
# Commit changes to database
session.commit()
tempo_fim = time.time()
print('### SQL INSERT ### ', df.shape[0], 'registros.')
print("Tempo de inserção: {:.5f} segundos".format((tempo_fim - tempo_ini)))

### SQL INSERT ###  1011250 registros.
Tempo de inserção: 168.50968 segundos


In [9]:
query1SQLite = []
for i in range(n):
    query = """SELECT * FROM coleta AS c WHERE c.regiao = 'SE';"""
    tempo_ini = time.time()
    result = pd.read_sql_query(query, session.bind)
    tempo_fim = time.time()
    print(i)
    print('### SQL - região sudeste ### ', result.shape[0], 'registro(s).')
    print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    query1SQLite.append(tempo_fim - tempo_ini)

0
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.21365 segundos
1
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.70700 segundos
2
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.66323 segundos
3
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.66170 segundos
4
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.82418 segundos
5
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.71001 segundos
6
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.68756 segundos
7
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.83674 segundos
8
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.69485 segundos
9
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.69311 segundos
10
### SQL - região sudeste ###  491866 registro(s).
Tempo de consulta: 3.82868 segundos
11
### SQL - região sudeste ###

In [10]:
query2SQLite = []
for i in range(n):
    query = """SELECT AVG(c.valor_venda) FROM coleta AS c ;"""
    tempo_ini = time.time()
    result = pd.read_sql_query(query, session.bind)
    tempo_fim = time.time()
    print(i)
    print('### SQL - preço médio de venda ### ', result.shape[0], 'registro(s).')
    print(result['AVG(c.valor_venda)'].values[0])
    print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    query2SQLite.append(tempo_fim - tempo_ini)

0
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.58700 segundos
1
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28354 segundos
2
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28309 segundos
3
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28400 segundos
4
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28400 segundos
5
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28400 segundos
6
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28300 segundos
7
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28400 segundos
8
### SQL - preço médio de venda ###  1 registro(s).
6.040825667265067
Tempo de consulta: 0.28312 segundos
9
### SQL - preço médio de venda ### 

In [11]:
query3SQLite = []
for i in range(n):
    query = """ SELECT c.regiao, AVG(c.valor_venda) FROM coleta AS c 
                GROUP BY c.regiao ORDER BY AVG(c.valor_venda) ASC;"""
    tempo_ini = time.time()
    result = pd.read_sql_query(query, session.bind)
    tempo_fim = time.time()
    print(i)
    print('### SQL - preço médio de venda agrupado por região ### ', result.shape[0], 'registro(s).')
    print(result)
    print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    query3SQLite.append(tempo_fim - tempo_ini)

0
### SQL - preço médio de venda agrupado por região ###  5 registro(s).
  regiao  AVG(c.valor_venda)
0     SE            5.905229
1     CO            6.051408
2      S            6.061496
3     NE            6.248689
4      N            6.395511
Tempo de consulta: 0.78000 segundos
1
### SQL - preço médio de venda agrupado por região ###  5 registro(s).
  regiao  AVG(c.valor_venda)
0     SE            5.905229
1     CO            6.051408
2      S            6.061496
3     NE            6.248689
4      N            6.395511
Tempo de consulta: 0.77600 segundos
2
### SQL - preço médio de venda agrupado por região ###  5 registro(s).
  regiao  AVG(c.valor_venda)
0     SE            5.905229
1     CO            6.051408
2      S            6.061496
3     NE            6.248689
4      N            6.395511
Tempo de consulta: 0.77545 segundos
3
### SQL - preço médio de venda agrupado por região ###  5 registro(s).
  regiao  AVG(c.valor_venda)
0     SE            5.905229
1     CO            

# Experimento Banco de Dados Documentos (MongoDB)

In [12]:
import pymongo

# Establish connection
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Create a database
db = client.precos
db.drop_collection(name_or_collection='coleta')

{'nIndexesWas': 1, 'ns': 'precos.coleta', 'ok': 1.0}

In [13]:
insertMongoDB = []
deleteMongoDB = []
for i in range(n):
    #Create objects
    tempo_ini = time.time()
    dictio = df.to_dict('records')
    db.coleta.insert_many(dictio)
    tempo_fim = time.time()
    print(i)
    print('### NoSQL INSERT ### ', df.shape[0], 'registros.')
    print("Tempo de inserção: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    insertMongoDB.append(tempo_fim - tempo_ini)
    #Drop collection
    tempo_ini = time.time()
    db.drop_collection(name_or_collection='coleta')
    tempo_fim = time.time()
    print(i)
    print('### NoSQL DELETE ### ')
    print("Tempo de delete: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    deleteMongoDB.append(tempo_fim - tempo_ini)

0
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 30.89822 segundos
0
### NoSQL DELETE ### 
Tempo de delete: 0.00100 segundos
1
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 29.98362 segundos
1
### NoSQL DELETE ### 
Tempo de delete: 0.00100 segundos
2
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 30.39232 segundos
2
### NoSQL DELETE ### 
Tempo de delete: 0.00100 segundos
3
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 30.77764 segundos
3
### NoSQL DELETE ### 
Tempo de delete: 0.00100 segundos
4
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 30.54740 segundos
4
### NoSQL DELETE ### 
Tempo de delete: 0.00100 segundos
5
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 29.84151 segundos
5
### NoSQL DELETE ### 
Tempo de delete: 0.00000 segundos
6
### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 31.16309 segundos
6
### NoSQL DELETE ### 
Tempo de delete: 0.00100 segundos
7
### NoSQL INSERT ###  1011250 re

In [14]:
#Create objects
tempo_ini = time.time()
dictio = df.to_dict('records')
db.coleta.insert_many(dictio)
tempo_fim = time.time()
print('### NoSQL INSERT ### ', df.shape[0], 'registros.')
print("Tempo de inserção: {:.5f} segundos".format((tempo_fim - tempo_ini)))

### NoSQL INSERT ###  1011250 registros.
Tempo de inserção: 30.44778 segundos


In [15]:
query1MongoDB = []
for i in range(n):
    find = {'Regiao - Sigla':'SE'}
    tempo_ini = time.time()
    count = db.coleta.count_documents(find)
    tempo_fim = time.time()
    print(i)
    print('### NoSQL - região sudeste ### ', count, 'registros.')
    print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    query1MongoDB.append(tempo_fim - tempo_ini)

0
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.81356 segundos
1
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78900 segundos
2
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78956 segundos
3
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.79057 segundos
4
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78900 segundos
5
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78800 segundos
6
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.79100 segundos
7
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.79058 segundos
8
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78800 segundos
9
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78554 segundos
10
### NoSQL - região sudeste ###  491866 registros.
Tempo de consulta: 0.78700 segundos
11
### NoSQL - região sudeste #

In [16]:
query2MongoDB = []
for i in range(n):
    tempo_ini = time.time()
    result = db.coleta.aggregate([{'$group': {'_id':'null', 'avg_val':{'$avg':'$Valor de Venda'}}}])
    tempo_fim = time.time()
    print(i)
    print('### NoSQL - preço médio de venda ### ')
    for r in result:
        print(r['avg_val'])
    print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    query2MongoDB.append(tempo_fim - tempo_ini)

0
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.03474 segundos
1
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01968 segundos
2
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01800 segundos
3
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01907 segundos
4
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01886 segundos
5
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.02200 segundos
6
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01997 segundos
7
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01900 segundos
8
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01814 segundos
9
### NoSQL - preço médio de venda ### 
6.040825667243511
Tempo de consulta: 1.01655 segundos
10
### NoSQL - preço médio de venda ### 
6.040825667243511
T

In [17]:
query3MongoDB = []
for i in range(n):
    tempo_ini = time.time()
    result = db.coleta.aggregate([{'$group': {'_id':'$Regiao - Sigla','avg_val':{'$avg':'$Valor de Venda'}}},
                                  {"$sort": {'avg_val': 1}}])
    tempo_fim = time.time()
    print(i)
    print('### NoSQL - preço médio de venda ### ')
    for r in result:
        print(r)
    print("Tempo de consulta: {:.5f} segundos".format((tempo_fim - tempo_ini)))
    query3MongoDB.append(tempo_fim - tempo_ini)

0
### NoSQL - preço médio de venda ### 
{'_id': 'SE', 'avg_val': 5.905229351083425}
{'_id': 'CO', 'avg_val': 6.051408198025571}
{'_id': 'S', 'avg_val': 6.061496396910155}
{'_id': 'NE', 'avg_val': 6.2486891971724825}
{'_id': 'N', 'avg_val': 6.395511331538896}
Tempo de consulta: 1.11300 segundos
1
### NoSQL - preço médio de venda ### 
{'_id': 'SE', 'avg_val': 5.905229351083425}
{'_id': 'CO', 'avg_val': 6.051408198025571}
{'_id': 'S', 'avg_val': 6.061496396910155}
{'_id': 'NE', 'avg_val': 6.2486891971724825}
{'_id': 'N', 'avg_val': 6.395511331538896}
Tempo de consulta: 1.10951 segundos
2
### NoSQL - preço médio de venda ### 
{'_id': 'SE', 'avg_val': 5.905229351083425}
{'_id': 'CO', 'avg_val': 6.051408198025571}
{'_id': 'S', 'avg_val': 6.061496396910155}
{'_id': 'NE', 'avg_val': 6.2486891971724825}
{'_id': 'N', 'avg_val': 6.395511331538896}
Tempo de consulta: 1.11354 segundos
3
### NoSQL - preço médio de venda ### 
{'_id': 'SE', 'avg_val': 5.905229351083425}
{'_id': 'CO', 'avg_val': 6.0514

# Resultados

In [18]:
experiment_results = pd.DataFrame()

In [19]:
experiment_results['INSERT SQLite'] = insertSQLite
experiment_results['INSERT MongoDB'] = insertMongoDB
experiment_results['DELETE SQLite'] = deleteSQLite
experiment_results['DELETE MongoDB'] = deleteMongoDB
experiment_results['QUERY 1 SQLite'] = query1SQLite
experiment_results['QUERY 1 MongoDB'] = query1MongoDB
experiment_results['QUERY 2 SQLite'] = query2SQLite
experiment_results['QUERY 2 MongoDB'] = query2MongoDB
experiment_results['QUERY 3 SQLite'] = query3SQLite
experiment_results['QUERY 3 MongoDB'] = query3MongoDB

In [20]:
experiment_results.loc['mean', 'INSERT SQLite'], experiment_results.loc['ci-inf', 'INSERT SQLite'], experiment_results.loc['ci-sup', 'INSERT SQLite'] = mean_confidence_interval(insertSQLite)
experiment_results.loc['mean', 'INSERT MongoDB'], experiment_results.loc['ci-inf', 'INSERT MongoDB'], experiment_results.loc['ci-sup', 'INSERT MongoDB'] = mean_confidence_interval(insertMongoDB)

experiment_results.loc['mean', 'DELETE SQLite'], experiment_results.loc['ci-inf', 'DELETE SQLite'], experiment_results.loc['ci-sup', 'DELETE SQLite'] = mean_confidence_interval(deleteSQLite)
experiment_results.loc['mean', 'DELETE MongoDB'], experiment_results.loc['ci-inf', 'DELETE MongoDB'], experiment_results.loc['ci-sup', 'DELETE MongoDB'] = mean_confidence_interval(deleteMongoDB)

experiment_results.loc['mean', 'QUERY 1 SQLite'], experiment_results.loc['ci-inf', 'QUERY 1 SQLite'], experiment_results.loc['ci-sup', 'QUERY 1 SQLite'] = mean_confidence_interval(query1SQLite)
experiment_results.loc['mean', 'QUERY 1 MongoDB'], experiment_results.loc['ci-inf', 'QUERY 1 MongoDB'], experiment_results.loc['ci-sup', 'QUERY 1 MongoDB'] = mean_confidence_interval(query1MongoDB)

experiment_results.loc['mean', 'QUERY 2 SQLite'], experiment_results.loc['ci-inf', 'QUERY 2 SQLite'], experiment_results.loc['ci-sup', 'QUERY 2 SQLite'] = mean_confidence_interval(query2SQLite)
experiment_results.loc['mean', 'QUERY 2 MongoDB'], experiment_results.loc['ci-inf', 'QUERY 2 MongoDB'], experiment_results.loc['ci-sup', 'QUERY 2 MongoDB'] = mean_confidence_interval(query2MongoDB)

experiment_results.loc['mean', 'QUERY 3 SQLite'], experiment_results.loc['ci-inf', 'QUERY 3 SQLite'], experiment_results.loc['ci-sup', 'QUERY 3 SQLite'] = mean_confidence_interval(query3SQLite)
experiment_results.loc['mean', 'QUERY 3 MongoDB'], experiment_results.loc['ci-inf', 'QUERY 3 MongoDB'], experiment_results.loc['ci-sup', 'QUERY 3 MongoDB'] = mean_confidence_interval(query3MongoDB)

experiment_results

,INSERT SQLite,INSERT MongoDB,DELETE SQLite,DELETE MongoDB,QUERY 1 SQLite,QUERY 1 MongoDB,QUERY 2 SQLite,QUERY 2 MongoDB,QUERY 3 SQLite,QUERY 3 MongoDB
0,168.546102,30.898222,0.173753,0.001000,3.213646,0.813561,0.586999,1.034739,0.779999,1.113001
1,167.379296,29.983619,0.175220,0.001000,3.707000,0.789000,0.283541,1.019685,0.776000,1.109515
2,167.152279,30.392316,0.174000,0.001001,3.663230,0.789558,0.283092,1.018003,0.775454,1.113542
3,166.854398,30.777635,0.171001,0.000999,3.661701,0.790568,0.284002,1.019067,0.775001,1.112543
4,167.080085,30.547401,0.168998,0.001002,3.824180,0.789000,0.283999,1.018863,0.771998,1.106997
5,167.602210,29.841508,0.173001,0.000000,3.710006,0.788000,0.283999,1.021999,0.785002,1.105931
6,166.889886,31.163087,0.171002,0.001000,3.687559,0.791001,0.283000,1.019972,0.782000,1.107550
7,168.385132,30.757841,0.170669,0.001001,3.836736,0.790577,0.284000,1.019000,0.778999,1.111544
8,166.704933,31.279717,0.171002,0.000999,3.694854,0.788002,0.283124,1.018140,0.775551,1.104000
9,167.505046,30.255647,0.170003,0.001000,3.693110,0.785539,0.284999,1.016546,0.772199,1.108998
